# Get gender

## Add `'gender'` key to database

In [4]:
import requests
from IPython.display import JSON
from gender_guesser_first_names import gender
from re import sub
import json
import pandas as pd

In [2]:
LOCAL=False
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [89]:
df[0]['name']

{'value': 'Albrecht, Hartwig', 'preferred_name': 'Hartwig Albrecht'}

In [90]:
D=gender.Detector(case_sensitive=False)

In [92]:
def force_get_gender(d):
    '''
     Get names from `name.value` and if empty get it from `full_name`
     and using `country` information if avalaible
    '''
    gender = 'unknown'
    names = ''
    if (d.get('name') and d.get('name').get('value') and 
        sub(r'\w\.','',d.get('name').get('value').split(', ')[-1]).strip()):
        names = sub(r'\w\.','',d.get('name').get('value').split(', ')[-1]).strip()
        names = names.replace('-',' ').strip()
    if not names and d.get('full_name'):
        names = sub(r'\w\.','',d.get('full_name').split(', ')[-1]).strip()
        names = names.replace('-',' ').strip()
    if names:
        if d.get('country'):
            gender = D.get_gender( names, country=d.get('country').strip() )
        else:
            gender = D.get_gender( names )
    return gender

In [93]:
kk = [d.update( {'gender': force_get_gender(d)} ) for d in df ]

In [5]:
TMP  = pd.DataFrame(df)
TMPU = TMP.drop_duplicates('author_id').reset_index(drop=True)

print(f'TOTAL: {TMP.shape[0]}')
print(f'Unique autors: {TMPU.shape[0]}')
print(f'''Results \
        male:  {TMPU[TMPU['gender']=='male'].shape[0]} + female: {
                TMPU[TMPU['gender']=='female'].shape[0]} + unknown: {
                TMPU[TMPU['gender']=='unknown'].shape[0]}.
         ''')
print(f'female ratio at probability: { 
               round(TMPU[TMPU['gender']=='female'].shape[0]/(TMPU[TMPU['gender']=='female'].shape[0] + TMPU[TMPU['gender']=='male'].shape[0]),2)*100  }%')

TOTAL: 57588
Unique autors: 29969
Results         male:  14755 + female: 2736 + unknown: 11855.
         
female ratio at probability: 16.0%


In [94]:
f=open('data/inspire_LA.json','w')
json.dump(df,f)
f.close()

## Analysis

In [95]:
import pandas as pd

In [96]:
DF = pd.read_json('data/inspire_LA.json')

Unique authors ids

In [97]:
DF.shape[0]

57588

In [98]:
DF = DF.drop_duplicates('author_id').reset_index(drop=True)
DF.shape[0]

29969

In [12]:
DF['first_names'] = DF['name'].str.get('value').astype(str).str.split(', ').str[-1]#.apply(
    #lambda s: sub('\w\.','',s)).str.replace('None','')

In [13]:
DF['first_names'] = DF.apply(lambda row: str(row['full_name']).split(', ')[-1]  
                             if row['first_names']=='None' 
                             else row['first_names'] ,axis = 'columns')

In [14]:
DF['gender'].unique()

array(['male', 'unknown', 'female', 'andy'], dtype=object)

In [15]:
print(DF['gender'].shape[0])
total = DF['gender'].dropna().shape[0]
total

29969


29969

In [17]:
male = DF[DF['gender'] == 'male'].shape[0]
female = DF[DF['gender'] == 'female'].shape[0]
unknown = DF[DF['gender'] == 'unknown'].shape[0]
unknown = unknown + DF[DF['gender'] == 'andy'].shape[0]

In [18]:
assert male + female + unknown == total

In [19]:
male+female

17491

In [20]:
male

14755

In [21]:
female

2736

In [22]:
round(female/male,2)

0.19

In [23]:
unknown

12478

In [24]:
DFFM = DF[ (( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFFM.shape

(17491, 14)

In [25]:
DFU = DF[ ~(( DF['gender'] == 'female') | ( DF['gender'] == 'male')) ]
DFU.shape

(12478, 14)

In [26]:
DFFM[['author_id','first_names','country','gender']].sample(50)

,author_id,first_names,country,gender
23889,M.B.D.Rocha.1,Mateus Broilo da,Brazil,male
23293,C.Frugiuele.1,Claudia,United States,female
28098,R.Rubim.1,Roberto,Brazil,male
20483,Y.Takahashi.1,Yasushi,Canada,male
15376,C.V.R.Board.1,Charles V.R.,United Kingdom,male
7433,M.R.Schreiber.1,Matthias R.,Chile,male
14325,Y.Verbin.1,Yosef,Israel,male
11484,E.Coleman.3,Evan,United States,male
11692,K.Louedec.1,Karim,France,male
19536,J.Tossa.1,Joel,Benin,male


In [27]:
#[['first_names','country','gender']]
DFU.sample(50)[['author_id','full_name','name','country','gender']]

,author_id,full_name,name,country,gender
22380,N.Fonseca.1,"Fonseca, Nayara","{'value': 'Fonseca, Nayara', 'preferred_name':...",Germany,unknown
1573,Sheng.Jiang.Zhu.1,"Zhu, S.J.","{'value': 'Zhu, Sheng-Jiang', 'native_names': ...",United States,andy
18378,I.Mejia.1,"Mejia, I.",None,United States,unknown
18619,E.Recillas.2,"Recillas, E.",None,Mexico,unknown
22895,A.M.Paredes.1,"Paredes, A.A.V.",None,Brazil,unknown
19053,P.F.Tavares.1,"Tavares, P.F.",None,Brazil,unknown
17164,A.Herrera.Aguilar.5,"Herrera-Aguilar, A.",None,Mexico,unknown
3860,S.Capozziello.2,"Capozziello, S.",None,Italy,unknown
27448,A.M.De.Lima.2,"De Lima, A.M.",None,Brazil,unknown
9739,A.Anabalon.2,"Anabalón, A.",None,Chile,unknown


check specific entries

In [28]:
JSON([d for d in df if d.get('author_id')=='F.Fontanot.3'][0])

<IPython.core.display.JSON object>

In [5]:
import requests
au = requests.get('https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1')
if au.status_code == 200:
    au = au.json()

In [7]:
len(au['hits']['hits'])

250

In [12]:
au['links']['next']

'https://inspirehep.net/api/authors/?sort=bestmatch&q=&size=250&page=2'

In [80]:
# https://github.com/inspirehep/rest-api-doc/issues/20#issuecomment-997932079
url='https://inspirehep.net/api/authors?sort=bestmatch&size=250&page=1&q=control_number:[1+TO+2852058]'
au = requests.get(url)
if au.status_code == 200:
    au = au.json()

In [81]:
len(au['hits']['hits'])

250

In [15]:
au.status_code

400

In [ ]:
au = requests.get(url)

In [58]:
step = 10000
ini = 981872
end = ini + step
for i in range(187):
    print(f'control_number:[{ini} TO {end}]')
    ini = end + 1
    end = ini + step

control_number:[981872 TO 991872]
control_number:[991873 TO 1001873]
control_number:[1001874 TO 1011874]
control_number:[1011875 TO 1021875]
control_number:[1021876 TO 1031876]
control_number:[1031877 TO 1041877]
control_number:[1041878 TO 1051878]
control_number:[1051879 TO 1061879]
control_number:[1061880 TO 1071880]
control_number:[1071881 TO 1081881]
control_number:[1081882 TO 1091882]
control_number:[1091883 TO 1101883]
control_number:[1101884 TO 1111884]
control_number:[1111885 TO 1121885]
control_number:[1121886 TO 1131886]
control_number:[1131887 TO 1141887]
control_number:[1141888 TO 1151888]
control_number:[1151889 TO 1161889]
control_number:[1161890 TO 1171890]
control_number:[1171891 TO 1181891]
control_number:[1181892 TO 1191892]
control_number:[1191893 TO 1201893]
control_number:[1201894 TO 1211894]
control_number:[1211895 TO 1221895]
control_number:[1221896 TO 1231896]
control_number:[1231897 TO 1241897]
control_number:[1241898 TO 1251898]
control_number:[1251899 TO 1261

In [74]:
au.status_code

200

In [77]:
len(au.json()['hits']['hits'])

250